In [1]:
import os, sys
parent_dir = os.path.abspath('../..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import

In [2]:
import time
from itertools import cycle
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.utils import make_grid
import torchvision.datasets as dset
import matplotlib.pyplot as plt

import utils_os
import mi


torch.manual_seed(time.time())

%load_ext autoreload
%autoreload

# Global Variables

In [3]:
DEVICE = 'cuda:0' 

class Hyperparams(utils_os.ConfigDict):

    def __init__(self): 
        self.device = DEVICE
        self.learning_rate = 1e-3
        self.n_epochs = 100
        self.grad_clip = None
        self.batch_size = 128
        self.image_shape = (32, 32)
        self.update_inner_every_num_epoch = 2

BASE_HYPERPARAMS = Hyperparams()

ROOT = '../..'
MODEL_SAVE_DIR = f'{ROOT}/results/c_s'
DATASET_DIR = f'{ROOT}/data'

NUM_COLOR_CHANNELS = 3
NUM_CLASSES = 9
NUM_DOMAIN = 2

DIM_Z_CONTENT, DIM_Z_DOMAIN = 128, 128

INFOMIN_TRAIN_BATCH_SIZE = 2500

# Prepare Dataloaders

### Align labels

In [4]:
"""
CIFAR
0: plane
1: car
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck

remove frog(6)
"""

"""
STL
0: plane
1: bird
2: car
3: cat
4: deer
5: dog
6: horse
7: monkey
8: ship
9: truck

remove monkey(7)
"""
CIFAR_MAP = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: -2,
    7: 6,
    8: 7,
    9: 8,
    -1: -1,
}

STL_MAP = {
    0: 0,
    1: 2,
    2: 1,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: -2,
    8: 7,
    9: 8,
    -1: -1,
}

### Load CIFAR10 and STL

In [5]:
if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)
    
# define data transformations
trans_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
])
trans_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(), 
])
target_trans_cifar = transforms.Compose([lambda x: CIFAR_MAP[x]])
target_trans_stl = transforms.Compose([lambda x: STL_MAP[x]])


# load cifar (remove the class 'frog')
m_train_set = dset.CIFAR10(root=DATASET_DIR, train=True, transform=trans_train, target_transform=target_trans_cifar, download=True)
m_test_set = dset.CIFAR10(root=DATASET_DIR, train=False, transform=trans_test, target_transform=target_trans_cifar, download=True)

cifar_train_indices = [idx for idx, (_, target) in enumerate(m_train_set) if target != -2]
m_train_loader = torch.utils.data.DataLoader(
                 dataset=torch.utils.data.Subset(m_train_set, cifar_train_indices),
                 batch_size=BASE_HYPERPARAMS.batch_size)
cifar_test_indices = [idx for idx, (_, target) in enumerate(m_test_set) if target != -2]
m_test_loader = torch.utils.data.DataLoader(
                dataset=torch.utils.data.Subset(m_test_set, cifar_test_indices),
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)
domain1 = 'CIFAR'

# load stl-10 (remove the class 'monkey')
mm_train_set = dset.STL10(root=DATASET_DIR, split='train', transform=trans_train, target_transform=target_trans_stl, download=True)
mm_test_set = dset.STL10(root=DATASET_DIR, split='test', transform=trans_test, target_transform=target_trans_stl, download=True)

stl_train_indices = [idx for idx, (_, target) in enumerate(mm_train_set) if target != -2]
mm_train_loader = torch.utils.data.DataLoader(
                 dataset=torch.utils.data.Subset(mm_train_set, stl_train_indices),
                 batch_size=BASE_HYPERPARAMS.batch_size)
stl_test_indices = [idx for idx, (_, target) in enumerate(mm_test_set) if target != -2]
mm_test_loader = torch.utils.data.DataLoader(
                dataset=torch.utils.data.Subset(mm_test_set, stl_test_indices),
                batch_size=BASE_HYPERPARAMS.batch_size,
                shuffle=False)  
domain2 = 'STL'

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Load all data into memeory for faster access when training infomin layers

In [6]:
def loader2tensor(dataloader):
    all_data, all_target = [], []
    for data, target in iter(dataloader):
        all_data.append(data)
        all_target.append(target)
    print('loader2tensor finished.', 'data max=', data.max(), 'data min=', data.min())
    return torch.cat(all_data, dim=0), torch.cat(all_target, dim=0)

all_data_m, all_label_m = loader2tensor(m_train_loader)
all_data_test_m, all_label_test_m = loader2tensor(m_test_loader)

all_data_mm, all_label_mm = loader2tensor(mm_train_loader)
all_data_test_mm, all_label_test_mm = loader2tensor(mm_test_loader)

loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)
loader2tensor finished. data max= tensor(1.) data min= tensor(0.)


# Experiments

In [7]:
from tasks.domain_adaptation.experiment import DomainAdaptation, train, test, exp_run


def inner_batch_provider():
    idx1, idx2 = torch.randperm(len(all_data_m)), torch.randperm(len(all_data_mm))
    infomin_x1, infomin_x2 = all_data_m[idx1[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE), all_data_mm[idx2[:INFOMIN_TRAIN_BATCH_SIZE]].to(DEVICE)

    return infomin_x1, infomin_x2


def get_model_path(hyperparams, epoch=None):
    if epoch is not None:
        return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}_{epoch}'
    return f'{MODEL_SAVE_DIR}/{hyperparams.estimator}/model_{hyperparams.get_name("alpha", "beta", "gamma")}'

In [8]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.3
hyperparams.n_slice = 100
hyperparams.learning_rate = 1e-3
hyperparams.batch_size = 256
hyperparams.estimator = 'SLICE'
hyperparams.n_epochs = 70

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device='cuda:0',
    model=da,
    # scheduler_func=lambda optimizer: torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 1.7737207362349605, loss_content: 1.1118791279658464, loss_domain: 0.634625044087289 redundancy: 0.09072184178348579  acc_d1: 59.333335876464844 acc_d2: 51.134342193603516 acc_domain: 70.70364379882812

epoch: 1, loss: 1.2871744179389846, loss_content: 0.8085724509937663, loss_domain: 0.4585331149504218 redundancy: 0.0668961846623832  acc_d1: 70.54444885253906 acc_d2: 59.15258026123047 acc_domain: 79.15740203857422

epoch: 2, loss: 1.1938913288250776, loss_content: 0.824276799047497, loss_domain: 0.34849690782352233 redundancy: 0.07039208360619738  acc_d1: 71.82221984863281 acc_d2: 59.942169189453125 acc_domain: 88.50044250488281

epoch: 3, loss: 1.0148279851591084, loss_content: 0.6515623990079047, loss_domain: 0.34350851052243947 redundancy: 0.06585690755606957  acc_d1: 77.47777557373047 acc_d2: 64.80204772949219 acc_domain: 85.01556396484375

epoch: 4, loss: 1.3116201556904215, loss_content: 0.4933339848484791, loss_domain: 0.8028707176866666 redundancy: 0.0513848494

In [9]:
import copy

hyperparams = copy.deepcopy(BASE_HYPERPARAMS)
hyperparams.alpha = 1.0
hyperparams.beta = 1.0
hyperparams.gamma = 0.005
hyperparams.estimator = 'CLUB'
hyperparams.learning_rate = 1e-3
hyperparams.inner_lr = 1e-3
hyperparams.inner_epochs = 5
hyperparams.inner_batch_size = 2500

da = DomainAdaptation(DIM_Z_CONTENT, DIM_Z_DOMAIN, NUM_CLASSES, NUM_DOMAIN, hyperparams).to(DEVICE)

da = exp_run(
    (m_train_loader, mm_train_loader), (m_test_loader, mm_test_loader),
    train, test,
    inner_batch_provider, get_model_path,
    hyperparams,
    device=DEVICE,
    model=da,
    scheduler_func=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
)

epoch: 0, loss: 0.19142541415254835, loss_content: 1.7954795394145266, loss_domain: 0.32359765637928334 redundancy: -385.5303671393596  acc_d1: 27.85555648803711 acc_d2: 20.30694007873535 acc_domain: 88.63383483886719

epoch: 1, loss: 1.5198287241895434, loss_content: 1.4633504353778464, loss_domain: 0.33159608320451117 redundancy: -55.023558952439  acc_d1: 45.22222137451172 acc_d2: 32.89590835571289 acc_domain: 87.8890609741211

epoch: 2, loss: 1.5666200980334215, loss_content: 1.091761806481321, loss_domain: 0.5264953727453527 redundancy: -10.327416561019252  acc_d1: 61.255558013916016 acc_d2: 51.490211486816406 acc_domain: 79.60760498046875

epoch: 3, loss: 1.0480088591575623, loss_content: 0.8353052592613328, loss_domain: 0.22685472761661235 redundancy: -2.83022489346249  acc_d1: 69.4000015258789 acc_d2: 56.35009002685547 acc_domain: 92.51333618164062

epoch: 4, loss: 1.0876953484306873, loss_content: 0.6993083009417628, loss_domain: 0.3943440771438706 redundancy: -1.19140665967699